# Megatonの使い方
Megaton の主要な使い方を Jupyter Notebook 上で順に体験できるデモです。各セルを上から実行していくと、認証からデータ取得、保存までの流れが確認できます。

## 0. 事前準備
- この Notebook はリポジトリと同じディレクトリに置いてください。
- 依存ライブラリが未インストールの場合はターミナルで `pip install -r requirements.txt` を実行します。
- サービスアカウント JSON を手元に用意し、Notebook からアクセスできるパスまたは環境変数に設定しておきます。

In [ ]:
# 1. 開発版 Megaton をインポートできるように設定します
%pip install -q -e /Users/maks/Repos/github_mak00s/megaton
%load_ext autoreload
%autoreload 2

Note: you may need to restart the kernel to use updated packages.


## 1. モジュールの読み込み
インストールされているモジュールの実体を確認しておきます。

In [ ]:
import megaton
import inspect
print(megaton.__file__)

/Users/maks/Repos/github_mak00s/megaton/megaton/__init__.py


## 2. 認証の準備
以下の 3 パターンを用意しました。自分の環境に合わせてコードを選択し、不要なセルはスキップまたはコメントアウトしてください。

### 2-1. JSON ファイルを直接参照する場合
`json_path` にサービスアカウント JSON のフルパスを指定します。

In [ ]:
from pathlib import Path
from megaton import start

json_path = Path('/path/to/service-account.json')  # TODO: ご自身のファイルパスに置き換えてください
json_path = Path('/Users/maks/Dropbox/python/key/sa-shiseido-corp-dts.json')
mg = start.Megaton(str(json_path))

### 2-2. Notebook 上で環境変数に読み込む場合
Notebook で JSON ファイルを読み込み、`os.environ` に動的に流し込む例です。

In [ ]:
# json_path = Path('/path/to/service-account.json')  # TODO: ご自身のファイルパスに置き換えてください
if json_path.exists():
    os.environ['MEGATON_CREDS_JSON'] = json_path.read_text()
    mg = start.Megaton()  # 環境変数を自動検出
else:
    print('サービスアカウント JSON が見つかりません:', temp_json_path)

### 2-3. JSON 文字列を直接渡す場合
環境変数や外部ストレージから JSON 文字列を取得し、`json_input` 引数に渡します。

In [ ]:
raw_json = json_path.read_text()
if raw_json:
    mg = start.Megaton(raw_json)
else:
    print('環境変数 MEGATON_CREDS_JSON が設定されていません')

### 2-4. OAuth2の場合

In [ ]:
json_path = Path('/Users/maks/Downloads/client_secret_python-selenium.json')
mg = start.Megaton(str(json_path))

KeyError: 'OAuth'

## 3. 認証確認
現在読み込まれているクレデンシャルが有効かチェックします。サービスアカウント JSON を読み込めていれば "Credentials valid: True" が表示されます。

In [ ]:
import json
from megaton import auth, constants

raw = os.environ.get('MEGATON_CREDS_JSON')
if raw:
    info = json.loads(raw)
    creds = auth.load_service_account_credentials_from_info(info, constants.DEFAULT_SCOPES)
    print('Credentials valid:', getattr(creds, 'valid', False))
else:
    print('認証情報が環境変数に設定されていません')


Credentials valid: True


## 4. Megaton ウィジェットの操作
上記いずれかの方法で `Megaton` インスタンス (`mg_*`) を生成すると、Notebook 上に認証ウィジェットが表示されます。アカウントやプロパティを選び、レポート実行・保存などのボタンを操作してください。

### 4-1. GA4 レポートの取得
`mg.report.ga4()` を呼び出し、プロパティ・ディメンション・指標を選択します。

In [ ]:
# 有効化されたGAのバージョン
print(mg.ga_ver)

# 有効化されたサービス
print(mg.enabled)

4


['ga4']

In [ ]:
# 権限を持つGA4アカウント
# print(mg.ga['4'].accounts)

# 選択されたGA4アカウントのID
mg.ga['4'].account.id

# 選択されたGA4アカウントに紐づくプロパティ
mg.ga['4'].account.properties

[{'id': '334854563',
  'name': '【新】資生堂企業情報サイト - GA4',
  'time_zone': 'Asia/Tokyo',
  'currency': 'JPY',
  'industry': 'BEAUTY_AND_FITNESS',
  'service_level': 'GOOGLE_ANALYTICS_STANDARD',
  'created_time': datetime.datetime(2022, 9, 29, 17, 39, 46, 402000, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>),
  'updated_time': datetime.datetime(2025, 5, 25, 22, 22, 39, 305000, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>),
  'data_retention': 'FOURTEEN_MONTHS',
  'data_retention_reset_on_activity': True}]

### 4-2. Google Sheets へ保存
取得結果を Google Sheets へ書き込む場合は `mg.save.to_gsheet()` を利用します。

In [ ]:
try:
    display(mg_inline.save.to_gsheet())
except NameError:
    print('まず Megaton インスタンスを用意してください')


### 4-3. BigQuery へ保存
BigQuery プロジェクトに権限がある場合、`mg.save.to_bigquery()` でテーブルに保存できます。

In [ ]:
try:
    display(mg_inline.save.to_bigquery())
except NameError:
    print('まず Megaton インスタンスを用意してください')


## 5. トラブルシュート
- 認証エラーが発生する場合は、サービスアカウントに必要な API が有効化されているか確認してください。
- GA4/BigQuery の権限が不足している場合は、対象プロパティやプロジェクトへの権限を付与します。
- ウィジェットが表示されない場合は Notebook / ブラウザを再読込してください。

---
必要に応じてセルをカスタマイズし、自分のワークフローに合わせたテンプレートノートブックとして利用してください。